<a href="https://colab.research.google.com/github/Anshlulla/Hindi-Summarization/blob/main/hindi_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rouge -qq

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from rouge import Rouge
import os
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)

True

In [ ]:
# Define the BiLSTM model
class BiLSTMSummarizer(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(BiLSTMSummarizer, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.encoder = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True, batch_first=True)
        self.decoder = nn.LSTM(embedding_dim, hidden_dim * 2, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size = src.shape[0]
        trg_len = trg.shape[1]
        trg_vocab_size = self.fc.out_features

        outputs = torch.zeros(batch_size, trg_len, trg_vocab_size).to(src.device)

        embedded = self.embedding(src)
        enc_output, (hidden, cell) = self.encoder(embedded)

        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1).unsqueeze(0)
        cell = torch.cat((cell[-2,:,:], cell[-1,:,:]), dim=1).unsqueeze(0)

        input = trg[:, 0]

        for t in range(1, trg_len):
            input_embedded = self.embedding(input).unsqueeze(1)
            output, (hidden, cell) = self.decoder(input_embedded, (hidden, cell))
            prediction = self.fc(output.squeeze(1))
            outputs[:, t] = prediction

            teacher_force = torch.rand(1).item() < teacher_forcing_ratio
            top1 = prediction.argmax(1)
            input = trg[:, t] if teacher_force else top1

        return outputs

In [ ]:
# Custom dataset class
class SummarizationDataset(Dataset):
    def __init__(self, articles, summaries, vocab, max_length=100):
        self.articles = articles
        self.summaries = summaries
        self.vocab = vocab
        self.max_length = max_length

    def __len__(self):
        return len(self.articles)

    def __getitem__(self, idx):
        article = self.articles[idx]
        summary = self.summaries[idx]

        article_indices = [self.vocab['<sos>']] + [self.vocab.get(token, self.vocab['<unk>']) for token in article][:self.max_length-2] + [self.vocab['<eos>']]
        summary_indices = [self.vocab['<sos>']] + [self.vocab.get(token, self.vocab['<unk>']) for token in summary][:self.max_length-2] + [self.vocab['<eos>']]

        article_indices = article_indices + [self.vocab['<pad>']] * (self.max_length - len(article_indices))
        summary_indices = summary_indices + [self.vocab['<pad>']] * (self.max_length - len(summary_indices))

        return torch.tensor(article_indices), torch.tensor(summary_indices)

In [ ]:
# Load and preprocess data
def load_data(file_path):
    df = pd.read_csv(file_path)
    return df['Content'].tolist(), df['Headline'].tolist()

# Tokenize text
def tokenize(text):
    return word_tokenize(text.lower())

# Build vocabulary
def build_vocab(texts, min_freq=2):
    word_freq = Counter()
    for text in texts:
        word_freq.update(text)

    vocab = {'<pad>': 0, '<unk>': 1, '<sos>': 2, '<eos>': 3}
    for word, freq in word_freq.items():
        if freq >= min_freq:
            vocab[word] = len(vocab)

    return vocab, {v: k for k, v in vocab.items()}

In [ ]:
# Load data
articles, summaries = load_data('/kaggle/input/hindi-news-dataset/hindi_news_dataset.csv')

# Tokenize data
tokenized_articles = [tokenize(article) for article in articles]
tokenized_summaries = [tokenize(summary) for summary in summaries]

# Build vocabulary
vocab, inv_vocab = build_vocab(tokenized_articles + tokenized_summaries)

# Split data
train_articles, test_articles, train_summaries, test_summaries = train_test_split(tokenized_articles, tokenized_summaries, test_size=0.2, random_state=42)
train_articles, val_articles, train_summaries, val_summaries = train_test_split(train_articles, train_summaries, test_size=0.1, random_state=42)

In [ ]:
# Create datasets
train_dataset = SummarizationDataset(train_articles, train_summaries, vocab)
val_dataset = SummarizationDataset(val_articles, val_summaries, vocab)
test_dataset = SummarizationDataset(test_articles, test_summaries, vocab)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Initialize model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BiLSTMSummarizer(len(vocab), 100, 128, len(vocab)).to(device)
device

device(type='cuda')

In [ ]:
# Train function
def train(model, iterator, optimizer, criterion, device, clip=1, teacher_forcing_ratio=0.5):
    model.train()
    epoch_loss = 0
    for batch in tqdm(iterator, desc="Training"):
        src, trg = batch
        src, trg = src.to(device), trg.to(device)

        optimizer.zero_grad()
        output = model(src, trg, teacher_forcing_ratio)

        output_dim = output.shape[-1]
        output = output[:, 1:].reshape(-1, output_dim)
        trg = trg[:, 1:].reshape(-1)

        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [ ]:
# Evaluation function
def evaluate(model, iterator, criterion, device):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for batch in tqdm(iterator, desc="Evaluating"):
            src, trg = batch
            src, trg = src.to(device), trg.to(device)

            output = model(src, trg, 0)  # turn off teacher forcing

            output_dim = output.shape[-1]
            output = output[:, 1:].reshape(-1, output_dim)
            trg = trg[:, 1:].reshape(-1)

            loss = criterion(output, trg)
            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [ ]:
def beam_search(model, src, vocab, inv_vocab, beam_width=3, max_length=100, min_length=10, device='cpu'):
    model.eval()
    with torch.no_grad():
        # Embedding the input sequence
        embedded = model.embedding(src)  # shape: (batch_size, seq_len, embedding_dim)
        enc_output, (hidden, cell) = model.encoder(embedded)  # LSTM encoder output

        # In case of bi-directional LSTM, combine the hidden states
        if model.encoder.bidirectional:
            hidden = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)  # shape: (batch_size, hidden_dim)
            cell = torch.cat((cell[-2, :, :], cell[-1, :, :]), dim=1)        # shape: (batch_size, hidden_dim)
        else:
            hidden = hidden[-1, :, :]  # Take the last layer if not bi-directional
            cell = cell[-1, :, :]      # Take the last layer if not bi-directional

        # Now we process one sequence at a time, so set batch size to 1
        hidden = hidden.unsqueeze(0)  # shape: (1, batch_size, hidden_dim)
        cell = cell.unsqueeze(0)      # shape: (1, batch_size, hidden_dim)

        # Initialize the beam with the start-of-sequence token
        beam = [([vocab['<sos>']], 0, hidden[:, 0:1, :], cell[:, 0:1, :])]  # Start with one sequence
        complete_hypotheses = []

        # Perform beam search
        for t in range(max_length):
            new_beam = []
            for seq, score, hidden, cell in beam:
                # If end-of-sequence token is reached and length is >= min_length, add to complete hypotheses
                if seq[-1] == vocab['<eos>'] and len(seq) >= min_length:
                    complete_hypotheses.append((seq, score))
                    continue

                # Prepare the input for the decoder (last predicted token)
                input = torch.LongTensor([seq[-1]]).unsqueeze(0).to(device)  # shape: (1, 1)
                input_embedded = model.embedding(input)  # shape: (1, 1, embedding_dim)

                # Pass through the decoder with the current hidden and cell states
                output, (hidden, cell) = model.decoder(input_embedded, (hidden, cell))  # hidden, cell are (1, 1, hidden_dim)
                predictions = model.fc(output.squeeze(1))  # shape: (1, vocab_size)

                # Prevent EOS if sequence is shorter than minimum length
                if len(seq) < min_length:
                    predictions[0][vocab['<eos>']] = float('-inf')

                # Get top beam_width predictions
                top_preds = torch.topk(predictions, beam_width, dim=1)

                # For each top prediction, extend the sequence and update the beam
                for i in range(beam_width):
                    new_seq = seq + [top_preds.indices[0][i].item()]
                    new_score = score - top_preds.values[0][i].item()  # Negative log probability
                    new_hidden = hidden.clone()
                    new_cell = cell.clone()
                    new_beam.append((new_seq, new_score, new_hidden, new_cell))

            # Sort by score and keep top beam_width sequences
            beam = sorted(new_beam, key=lambda x: x[1])[:beam_width]

            if len(complete_hypotheses) >= beam_width:
                break

        # Sort and return the best sequence
        complete_hypotheses = sorted(complete_hypotheses, key=lambda x: x[1])
        if complete_hypotheses:
            best_seq = complete_hypotheses[0][0]
        else:
            best_seq = beam[0][0]

    # Convert sequence of indices back to words
    return [inv_vocab[idx] for idx in best_seq if idx not in [vocab['<sos>'], vocab['<eos>'], vocab['<pad>']]]

In [ ]:
# Save model function
def save_model(model, vocab, filepath, best_val_loss):
    torch.save({
        'model_state_dict': model.state_dict(),
        'vocab': vocab,
        'best_val_loss': best_val_loss # Save best_val_loss
    }, filepath)
    print(f"Model saved to {filepath}")

In [ ]:
# Define optimizer
learning_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss(ignore_index=vocab['<pad>'])

# Clear unused GPU memory
torch.cuda.empty_cache()

# Define learning rate scheduler (CosineAnnealingLR)
# It will reduce the learning rate when validation loss stops improving
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=2, verbose=True)

/opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [ ]:
# Training loop
# 1st epoch
num_epochs = 1
best_val_loss = float('inf')
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, optimizer, criterion, device)
    val_loss = evaluate(model, val_loader, criterion, device)
    print(f'Epoch: {epoch+1:02}')
    print(f'\tTrain Loss: {train_loss:.3f}')
    print(f'\t Val. Loss: {val_loss:.3f}')

    # Step the scheduler based on the validation loss
    scheduler.step(val_loss)

    # Save model if validation loss improves
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        save_model(model, vocab, 'best_model.pth', best_val_loss)
        torch.cuda.empty_cache() # Clear unused cache to free up GPU memory

Evaluating: 100%|██████████| 464/464 [04:05<00:00,  1.89it/s]
/opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:232: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Epoch: 01
	Train Loss: 6.030
	 Val. Loss: 5.695
Model saved to best_model.pth


In [ ]:
# Load model function
def load_model(filepath, device):
    checkpoint = torch.load(filepath, map_location=device)
    vocab = checkpoint['vocab']
    best_val_loss = checkpoint['best_val_loss'] # Load best_val_loss
    model = BiLSTMSummarizer(len(vocab), 100, 128, len(vocab)).to(device)
    model.load_state_dict(checkpoint['model_state_dict'])
    return model, checkpoint, best_val_loss # Return best_val_loss

In [ ]:
# Load the best model after 1st epoch
best_model, _, best_val_loss = load_model('/kaggle/working/best_model.pth', device)

# Training for 2nd and 3rd epochs
num_epochs = 2

# Train for the second epoch using the saved model
for epoch in range(num_epochs):
    train_loss = train(best_model, train_loader, optimizer, criterion, device)
    val_loss = evaluate(best_model, val_loader, criterion, device)

    # Print train and validation loss
    print(f'Epoch: {epoch+1:02}')
    print(f'\tTrain Loss: {train_loss:.3f}')
    print(f'\t Val. Loss: {val_loss:.3f}')

    # Save the model if the validation loss improves
    if val_loss < best_val_loss:
        best_val_loss = val_loss  # Update the best validation loss
        save_model(best_model, vocab, 'best_model.pth', best_val_loss)  # Save model
        torch.cuda.empty_cache() # Clear unused cache to free up GPU memory

# After training is complete
torch.cuda.empty_cache()  # Clear GPU cache one last time

/tmp/ipykernel_30/2281548745.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(filepath, map_location=device)
Evaluating: 100%|██████████| 464/464 

Epoch: 01
	Train Loss: 5.058
	 Val. Loss: 5.695


Evaluating: 100%|██████████| 464/464 [04:04<00:00,  1.90it/s]

Epoch: 02
	Train Loss: 5.060
	 Val. Loss: 5.695


In [ ]:
# Load the best model for testing
best_model, _, _ = load_model('best_model.pth', device)

# Test the model
test_loss = evaluate(best_model, test_loader, criterion, device)
print(f'Test Loss: {test_loss:.3f}')

# Evaluate using ROUGE score
rouge = Rouge()
best_model.eval()
predictions = []
references = []
with torch.no_grad():
    for batch in tqdm(test_loader, desc="Generating summaries"):
        src, trg = batch
        src = src.to(device)
        pred = beam_search(best_model, src, vocab, inv_vocab, min_length=10, device=device)  # Set minimum length
        predictions.extend([' '.join(pred)])
        references.extend([' '.join([inv_vocab[idx.item()] for idx in trg[0] if idx.item() not in [vocab['<sos>'], vocab['<eos>'], vocab['<pad>']]])])

# Ensure all predictions meet the minimum length
min_length = 10  # Set this to your desired minimum length
predictions = [p if len(p.split()) >= min_length else p + ' ' + ' '.join(['<pad>'] * (min_length - len(p.split()))) for p in predictions]

scores = rouge.get_scores(predictions, references, avg=True)
print("ROUGE scores:")
print(scores)

/tmp/ipykernel_30/2281548745.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(filepath, map_location=device)
Evaluating: 100%|██████████| 1160/116

Test Loss: 5.700


Generating summaries: 100%|██████████| 1160/1160 [00:46<00:00, 24.97it/s]


ROUGE scores:
{'rouge-1': {'r': 0.23247931357899754, 'p': 0.4455709328985187, 'f': 0.29887501308560693}, 'rouge-2': {'r': 0.06618892052101748, 'p': 0.09416376726721573, 'f': 0.07608365108299221}, 'rouge-l': {'r': 0.2067549893741905, 'p': 0.3964412933809486, 'f': 0.2657179110762273}}


In [ ]:
print("Loading pre-trained model...")
trained_model, checkpoint, _ = load_model('best_model.pth', device)
vocab = checkpoint['vocab']
inv_vocab = {v: k for k, v in vocab.items()}
trained_model = trained_model.to(device)

Loading pre-trained model...


/tmp/ipykernel_30/2281548745.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(filepath, map_location=device)


In [ ]:
# Modified Summarization bot
def summarize_text(model, vocab, inv_vocab, text, max_length=100, min_length=10, beam_width=3, device='cpu', debug=False):
    model.eval()
    tokens = tokenize(text)[:max_length]
    indices = [vocab['<sos>']] + [vocab.get(token, vocab['<unk>']) for token in tokens] + [vocab['<eos>']]
    src = torch.LongTensor(indices).unsqueeze(0).to(device)

    summary = beam_search(model, src, vocab, inv_vocab, beam_width, max_length, min_length, device)

    if debug:
        print("Input tokens:", tokens)
        print("Input indices:", indices)
        print("Generated indices:", [vocab[word] for word in summary])
        print("Summary length:", len(summary))

    return ' '.join(summary)

In [ ]:
# Example usage of the summarization bot
input_text = "ऑस्ट्रेलिया ने ब्लूमफोनटीन में पहले वनडे में दक्षिण अफ्रीका को 3-विकेट से हरा दिया। यह 12 वर्षों में दक्षिण अफ्रीका के खिलाफ उसकी धरती पर ऑस्ट्रेलिया की पहली वनडे जीत है। ऑस्ट्रेलिया का स्कोर 16.3 ओवर में 113/7 था लेकिन मार्नस लबुशेन और ऐश्टन एगर की 112* रनों की साझेदारी की बदौलत उसने 40.2 ओवर में लक्ष्य हासिल कर लिया।"
summary = summarize_text(trained_model, vocab, inv_vocab, input_text, min_length=10, device=device, debug=True)
print("Generated Summary:")
print(summary)
print("Summary length:", len(summary.split()))

Input tokens: ['ऑस्ट्रेलिया', 'ने', 'ब्लूमफोनटीन', 'में', 'पहले', 'वनडे', 'में', 'दक्षिण', 'अफ्रीका', 'को', '3-विकेट', 'से', 'हरा', 'दिया।', 'यह', '12', 'वर्षों', 'में', 'दक्षिण', 'अफ्रीका', 'के', 'खिलाफ', 'उसकी', 'धरती', 'पर', 'ऑस्ट्रेलिया', 'की', 'पहली', 'वनडे', 'जीत', 'है।', 'ऑस्ट्रेलिया', 'का', 'स्कोर', '16.3', 'ओवर', 'में', '113/7', 'था', 'लेकिन', 'मार्नस', 'लबुशेन', 'और', 'ऐश्टन', 'एगर', 'की', '112*', 'रनों', 'की', 'साझेदारी', 'की', 'बदौलत', 'उसने', '40.2', 'ओवर', 'में', 'लक्ष्य', 'हासिल', 'कर', 'लिया।']
Input indices: [2, 4910, 37, 8412, 14, 193, 7443, 14, 2425, 2426, 10, 8413, 86, 7167, 96, 67, 1330, 1254, 14, 2425, 2426, 12, 189, 274, 8414, 50, 4910, 8, 725, 7443, 2940, 35, 4910, 71, 7770, 8415, 7479, 14, 8416, 428, 596, 7627, 7628, 43, 8417, 8418, 8, 8419, 7930, 8, 3882, 8, 7536, 434, 8420, 7479, 14, 4471, 4275, 20, 1960, 3]
Generated indices: [237, 37, 7443, 1249, 7458, 3979, 14, 237, 12, 183, 237, 12, 93, 15120, 86, 7751]
Summary length: 16
Generated Summary:
भारत ने वनडे व